<a href="https://colab.research.google.com/github/daisyKim12/Tensorflow_Study/blob/main/Lecture_C3_cat_and_dog_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Category 3
Image Classification using transferin **VGG16** model

Computer Vision with CNNs
<br>
<br>For this exercise you will build a cats v dogs classifier
<br>using the Cats v Dogs dataset from TFDS.
<br>Be sure to use the final layer as shown
<br>    **(Dense, 2 neurons, softmax activation)**
<br>
<br>The testing infrastructre will **resize all images to 224x224**
<br>with **3 bytes of color depth**. Make sure your input layer trains

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

In [ ]:
dataset_name = 'cats_vs_dogs'

train_dataset = tfds.load(name = dataset_name, split='train[:80%]')
valid_dataset = tfds.load(name = dataset_name, split='train[80%:]')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete55W9GP/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [ ]:
for data in train_dataset.take(1):
  x = data['image']
  y = data['label']
  x = x /255
  x = tf.image.resize(x, size=(244, 244))
  print(x)

tf.Tensor(
[[[0.94683164 0.97036105 0.97036105]
  [0.9283489  0.9467674  0.94421196]
  [0.938734   0.95374525 0.94914865]
  ...
  [0.7627295  0.7068521  0.5235387 ]
  [0.67951995 0.62753874 0.44176465]
  [0.6202961  0.57346374 0.3851152 ]]

 [[0.9337512  0.95641273 0.95597875]
  [0.9260336  0.9441498  0.9414432 ]
  [0.9122109  0.92722213 0.92262554]
  ...
  [0.7692894  0.7148214  0.53197604]
  [0.71466976 0.6676305  0.48009235]
  [0.6978315  0.65778005 0.46638072]]

 [[0.9297923  0.94547856 0.941557  ]
  [0.91253185 0.9282181  0.92429656]
  [0.86094123 0.87661344 0.8700339 ]
  ...
  [0.7828922  0.7333583  0.5561645 ]
  [0.7812515  0.7358159  0.5534096 ]
  [0.77204967 0.74041986 0.5476969 ]]

 ...

 [[0.5778909  0.54822195 0.36305645]
  [0.6235258  0.5999964  0.40665016]
  [0.82276    0.81525433 0.61424184]
  ...
  [0.43779472 0.44805074 0.33361632]
  [0.51615965 0.53928167 0.4222775 ]
  [0.32614666 0.36581963 0.24943341]]

 [[0.5939511  0.5642822  0.37911662]
  [0.5304507  0.50692123 0

In [ ]:
def preprocess(data):
    # x, y 데이터를 정의합니다.
    x = data['image']
    y = data['label']
    # image 정규화(Normalization)
    x = x / 255
    # 사이즈를 (224, 224)로 변환합니다.
    x = tf.image.resize(x, size=(224, 224))
    # x, y  데이터를 return 합니다.
    return x, y

In [ ]:
batch_size=32

In [ ]:
train_data = train_dataset.map(preprocess).batch(batch_size)
valid_data = valid_dataset.map(preprocess).batch(batch_size)

#Use Transfer Model

In [ ]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
transfer_model.trainable=False

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
model = Sequential([
    transfer_model,
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax'),
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 27,626,178
Trainable params: 12,911,490
No

#Compile, Set Checkpoint, Fit, Load Weights

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

In [ ]:
model.fit(train_data,
          validation_data=(valid_data),
          epochs=20,
          callbacks=[checkpoint],
          )

Epoch 1/20
582/582 [==============================] - ETA: 0s - loss: 0.3002 - acc: 0.8858
Epoch 1: val_loss improved from inf to 0.18376, saving model to my_checkpoint.ckpt
582/582 [==============================] - 108s 160ms/step - loss: 0.3002 - acc: 0.8858 - val_loss: 0.1838 - val_acc: 0.9205
Epoch 2/20
582/582 [==============================] - ETA: 0s - loss: 0.1809 - acc: 0.9240
Epoch 2: val_loss improved from 0.18376 to 0.16119, saving model to my_checkpoint.ckpt
582/582 [==============================] - 91s 156ms/step - loss: 0.1809 - acc: 0.9240 - val_loss: 0.1612 - val_acc: 0.9334
Epoch 3/20
171/582 [=======>......................] - ETA: 51s - loss: 0.1585 - acc: 0.9333

KeyboardInterrupt: ignored

In [ ]:
model.load_weights(checkpoint_path)